# Exploratory Data Analysis of Rossmann Pharmaceuticals Sales Data

## Imports

In [133]:
import numpy as np
import pandas as pd
import math
import sys, os

In [134]:
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from csv_helper import CsvHelper
from df_selector import *
from df_cleaner import *
from df_visualizer import *
from df_outlier_handler import OutlierHandler

In [135]:
pd.set_option('display.max_columns', 50)

## Utility Functions

In [136]:
def unique_values_df(df):
    unique_values = {'Column': [], 'Unique values': []}
    for col in df:
        unique_values['Column'].append(col)
        values = df[col].value_counts().index.to_list()
        unique_values['Unique values'].append(values)
        tmp = pd.DataFrame(unique_values)
    return tmp

## Reading Data

In [137]:
# create a CsvHelper object
csv_helper = CsvHelper()

In [138]:
# reading the store csv file
store_df = csv_helper.read_csv("../data/store.csv")
store_df.head(10)

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN
5,6,a,a,310.0,12.0,2013.0,0,NaN,NaN,NaN
6,7,a,c,24000.0,4.0,2013.0,0,NaN,NaN,NaN
7,8,a,a,7520.0,10.0,2014.0,0,NaN,NaN,NaN
8,9,a,c,2030.0,8.0,2000.0,0,NaN,NaN,NaN
9,10,a,a,3160.0,9.0,2009.0,0,NaN,NaN,NaN


In [139]:
# reading the sales training csv file
train_df = csv_helper.read_csv("../data/train.csv")
train_df.head(10)

/home/z/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3361: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1
5,6,5,2015-07-31,5651,589,1,1,0,1
6,7,5,2015-07-31,15344,1414,1,1,0,1
7,8,5,2015-07-31,8492,833,1,1,0,1
8,9,5,2015-07-31,8565,687,1,1,0,1
9,10,5,2015-07-31,7185,681,1,1,0,1


## General Statistics

### store_df

In [140]:
# number of elements in the store df
store_df.size

11150

In [141]:
# rows and columns in the df
store_df.shape

(1115, 10)

In [142]:
store_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1115 entries, 0 to 1114
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Store                      1115 non-null   int64  
 1   StoreType                  1115 non-null   object 
 2   Assortment                 1115 non-null   object 
 3   CompetitionDistance        1112 non-null   float64
 4   CompetitionOpenSinceMonth  761 non-null    float64
 5   CompetitionOpenSinceYear   761 non-null    float64
 6   Promo2                     1115 non-null   int64  
 7   Promo2SinceWeek            571 non-null    float64
 8   Promo2SinceYear            571 non-null    float64
 9   PromoInterval              571 non-null    object 
dtypes: float64(5), int64(2), object(3)
memory usage: 87.2+ KB


### train_df

In [143]:
# number of elements in the train df
train_df.size

9154881

In [144]:
# rows and columns in the df
train_df.shape

(1017209, 9)

In [145]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   Store          1017209 non-null  int64 
 1   DayOfWeek      1017209 non-null  int64 
 2   Date           1017209 non-null  object
 3   Sales          1017209 non-null  int64 
 4   Customers      1017209 non-null  int64 
 5   Open           1017209 non-null  int64 
 6   Promo          1017209 non-null  int64 
 7   StateHoliday   1017209 non-null  object
 8   SchoolHoliday  1017209 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 69.8+ MB


## Missing Values

### store_df

In [146]:
percent_missing_values(store_df)

The dataset contains 21.01 % missing values.


In [147]:
missing_df = missing_values_table(store_df)

Your selected dataframe has 10 columns.
There are 6 columns that have missing values.


In [148]:
missing_df

,Missing Values,% of Total Values,Dtype
Promo2SinceWeek,544,48.79,float64
Promo2SinceYear,544,48.79,float64
PromoInterval,544,48.79,object
CompetitionOpenSinceMonth,354,31.75,float64
CompetitionOpenSinceYear,354,31.75,float64
CompetitionDistance,3,0.27,float64


From the missing table we can recognize two groups with a pattern, the first 3 columns and the next 2 columns. Columns in each group have the same number of missing values. This indicates that if we investigate deeper we might get some insight.

Let's see the row number of rows with missing values in general then in each group.

In [149]:
# number of rows with missing values for the whole dataset
count_missing_rows(store_df)

750 rows(67.26%) contain atleast one missing value.


In [150]:
# number of rows with missing values for group 1 (Promo2SinceWeek, Promo2SinceYear, and PromoInterval)
count_missing_rows(store_df[['Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval']])

544 rows(48.79%) contain atleast one missing value.


In [151]:
# number of rows with missing values for group 2 (CompetitionOpenSinceMonth, and CompetitionOpenSinceYear)
count_missing_rows(store_df[['CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear']])

354 rows(31.75%) contain atleast one missing value.


As guessed we have got an interesting insight. Each columuns in these groups have missing values in the same rows as their groupmates. These means the missing values in these groups appear at the same time as a cluster. From the column names and the data fields' description in the challange document we can understand that the values of the columns in each group share a common information. Let's investigate.

In [152]:
# dataframe containing the missing rows for columns in group 1
group1_df = store_df[store_df['Promo2SinceWeek'].isna()]
group1_df.head(10)

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN
5,6,a,a,310.0,12.0,2013.0,0,NaN,NaN,NaN
6,7,a,c,24000.0,4.0,2013.0,0,NaN,NaN,NaN
7,8,a,a,7520.0,10.0,2014.0,0,NaN,NaN,NaN
8,9,a,c,2030.0,8.0,2000.0,0,NaN,NaN,NaN
9,10,a,a,3160.0,9.0,2009.0,0,NaN,NaN,NaN
15,16,a,c,3270.0,NaN,NaN,0,NaN,NaN,NaN
22,23,d,a,4060.0,8.0,2005.0,0,NaN,NaN,NaN


We can see that the Promo2 column is 0 for the first 10 rows. From intution we can infer that if there is no promotion the values for the columns in gropu 1(Promo2SinceWeek, Promo2SinceYear, and PromoInterval) should be null. Let's prove it.

In [153]:
# all unique values for each column
unique_values_df(group1_df)

,Column,Unique values
0,Store,"[1024, 687, 672, 676, 679, 682, 683, 686, 689,..."
1,StoreType,"[a, d, c, b]"
2,Assortment,"[a, c, b]"
3,CompetitionDistance,"[50.0, 140.0, 250.0, 30.0, 720.0, 2640.0, 110...."
4,CompetitionOpenSinceMonth,"[9.0, 4.0, 11.0, 3.0, 10.0, 12.0, 7.0, 6.0, 2...."
5,CompetitionOpenSinceYear,"[2013.0, 2012.0, 2014.0, 2009.0, 2010.0, 2005...."
6,Promo2,[0]
7,Promo2SinceWeek,[]
8,Promo2SinceYear,[]
9,PromoInterval,[]


In [154]:
group1_df['Promo2'].value_counts()

0    544
Name: Promo2, dtype: int64

All 544 values are 0. Now let's check the count of 0 in the Promo2 column for the whole store dataset.

In [155]:
store_df['Promo2'].value_counts()

1    571
0    544
Name: Promo2, dtype: int64

Yes, we are right. All rows with 0 value in their Promo2 columns have null values in the columns Promo2SinceWeek, Promo2SinceYear, and PromoInterval.

Now let's impute the missing values in the columns Promo2SinceWeek and Promo2SinceYear with 0 to convey the meaning of absence since year and week can't be 0. For PromoInterval we should dig deeper what kind of data it holds since its data type is object.

In [156]:
# check the exact data type of the object
pd.api.types.infer_dtype(store_df['PromoInterval'])

'string'

In [157]:
# all the unique values 
store_df['PromoInterval'].value_counts()

Jan,Apr,Jul,Oct     335
Feb,May,Aug,Nov     130
Mar,Jun,Sept,Dec    106
Name: PromoInterval, dtype: int64

For the column PromoInterval, we will impute it with '0,0,0,0' as the other values follow list-type format containing four months. So inorder to impute these columns we should first investigate if the values we will insert exist in each column as this will oppose our intention.

In [158]:
store_df[store_df['Promo2SinceWeek'] == 0].shape

(0, 10)

In [159]:
store_df[store_df['Promo2SinceYear'] == 0].shape

(0, 10)

In [160]:
store_df[store_df['PromoInterval'] == '0,0,0,0'].shape

(0, 10)

Since the values don't appear in these columns, let's fill the missing values by them.

In [161]:
fix_missing_value(store_df, ['Promo2SinceWeek', 'Promo2SinceYear'], 0)

544 missing values in the column Promo2SinceWeek have been replaced by 0.
544 missing values in the column Promo2SinceYear have been replaced by 0.


In [162]:
fix_missing_value(store_df, ['PromoInterval'], '0,0,0,0')

544 missing values in the column PromoInterval have been replaced by 0,0,0,0.


Now it's time for the second group (i.e CompetitionOpenSinceMonth and CompetitionOpenSinceYear).

In [163]:
# dataframe containing the missing rows for columns in group 1
group2_df = store_df[store_df['CompetitionOpenSinceMonth'].isna()]
group2_df.head(10)

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
11,12,a,c,1070.0,NaN,NaN,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
12,13,d,a,310.0,NaN,NaN,1,45.0,2009.0,"Feb,May,Aug,Nov"
15,16,a,c,3270.0,NaN,NaN,0,0.0,0.0,"0,0,0,0"
18,19,a,c,3240.0,NaN,NaN,1,22.0,2011.0,"Mar,Jun,Sept,Dec"
21,22,a,a,1040.0,NaN,NaN,1,22.0,2012.0,"Jan,Apr,Jul,Oct"
25,26,d,a,2300.0,NaN,NaN,0,0.0,0.0,"0,0,0,0"
28,29,d,c,2170.0,NaN,NaN,0,0.0,0.0,"0,0,0,0"
31,32,a,a,2910.0,NaN,NaN,1,45.0,2009.0,"Feb,May,Aug,Nov"
39,40,a,a,180.0,NaN,NaN,1,45.0,2009.0,"Feb,May,Aug,Nov"
40,41,d,c,1180.0,NaN,NaN,1,31.0,2013.0,"Jan,Apr,Jul,Oct"


In [164]:
# all unique values for each column in the dataframe
unique_values_df(group2_df)

,Column,Unique values
0,Store,"[512, 392, 855, 353, 352, 350, 617, 348, 346, ..."
1,StoreType,"[a, d, c, b]"
2,Assortment,"[c, a, b]"
3,CompetitionDistance,"[250.0, 50.0, 2410.0, 350.0, 130.0, 90.0, 220...."
4,CompetitionOpenSinceMonth,[]
5,CompetitionOpenSinceYear,[]
6,Promo2,"[1, 0]"
7,Promo2SinceWeek,"[0.0, 40.0, 14.0, 45.0, 31.0, 22.0, 5.0, 35.0,..."
8,Promo2SinceYear,"[0.0, 2013.0, 2011.0, 2009.0, 2012.0, 2014.0, ..."
9,PromoInterval,"[0,0,0,0, Jan,Apr,Jul,Oct, Feb,May,Aug,Nov, Ma..."


I can't see any pattern with the other columns. But from the column names and the data fields' description in the challange document we can understand that if CompetitionOpenSinceMonth and CompetitionOpenSinceYear are null it means there was already a competition when the store was opened. So we need impute the missing values in a meaningful way. Before doing that we will hold this meaning of an already existing competition in a separate column named CompetitionBeforeStoreOpened so that we can make use of this information in our analysis and prediction. This column will hold values of 0 and 1, 0 indicating the absence of competition at the time the store was opened and 1 indicating the presence of competition at the time the store was opened.

In [165]:
# deriving new column
store_df['CompetitionBeforeStoreOpened'] = store_df['CompetitionOpenSinceYear'].apply(lambda x: 1 if np.isnan(x) else 0)
store_df

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,CompetitionBeforeStoreOpened
0,1,c,a,1270.0,9.0,2008.0,0,0.0,0.0,"0,0,0,0",0
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",0
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",0
3,4,c,c,620.0,9.0,2009.0,0,0.0,0.0,"0,0,0,0",0
4,5,a,a,29910.0,4.0,2015.0,0,0.0,0.0,"0,0,0,0",0
...,...,...,...,...,...,...,...,...,...,...,...
1110,1111,a,a,1900.0,6.0,2014.0,1,31.0,2013.0,"Jan,Apr,Jul,Oct",0
1111,1112,c,c,1880.0,4.0,2006.0,0,0.0,0.0,"0,0,0,0",0
1112,1113,a,c,9260.0,NaN,NaN,0,0.0,0.0,"0,0,0,0",1
1113,1114,a,c,870.0,NaN,NaN,0,0.0,0.0,"0,0,0,0",1


Now we can impute the columns CompetitionOpenSinceMonth and CompetitionOpenSinceYear. So we will impute missing values in the column CompetitionOpenSinceYear and CompetitionOpenSinceMonth with its minimun value and 1 respectively. This kind of imputing assumes that the stores with missing values in these columns were opened after the first month of the minimum year value in the column CompetitionOpenSinceYear.

Let's find the minimum year value for CompetitionOpenSinceYear and impute the missing values.

In [166]:
min_year = store_df['CompetitionOpenSinceYear'].min()
min_year

1900.0

In [167]:
# impute the column CompetitionOpenSinceYear with 1900 and the column CompetitionOpenSinceMonth with 1
fix_missing_value(store_df, ['CompetitionOpenSinceYear'], min_year)
fix_missing_value(store_df, ['CompetitionOpenSinceMonth'], 1)

354 missing values in the column CompetitionOpenSinceYear have been replaced by 1900.0.
354 missing values in the column CompetitionOpenSinceMonth have been replaced by 1.


In [168]:
store_df

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,CompetitionBeforeStoreOpened
0,1,c,a,1270.0,9.0,2008.0,0,0.0,0.0,"0,0,0,0",0
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",0
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",0
3,4,c,c,620.0,9.0,2009.0,0,0.0,0.0,"0,0,0,0",0
4,5,a,a,29910.0,4.0,2015.0,0,0.0,0.0,"0,0,0,0",0
...,...,...,...,...,...,...,...,...,...,...,...
1110,1111,a,a,1900.0,6.0,2014.0,1,31.0,2013.0,"Jan,Apr,Jul,Oct",0
1111,1112,c,c,1880.0,4.0,2006.0,0,0.0,0.0,"0,0,0,0",0
1112,1113,a,c,9260.0,1.0,1900.0,0,0.0,0.0,"0,0,0,0",1
1113,1114,a,c,870.0,1.0,1900.0,0,0.0,0.0,"0,0,0,0",1


Now we are left with missing values in the column CompetitionDistance. Let's see the rows containing these missing values.

In [169]:
temp_df = store_df[store_df['CompetitionDistance'].isna()]
temp_df

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,CompetitionBeforeStoreOpened
290,291,d,a,NaN,1.0,1900.0,0,0.0,0.0,"0,0,0,0",1
621,622,a,c,NaN,1.0,1900.0,0,0.0,0.0,"0,0,0,0",1
878,879,d,a,NaN,1.0,1900.0,1,5.0,2013.0,"Feb,May,Aug,Nov",1


In [170]:
unique_values_df(temp_df)

,Column,Unique values
0,Store,"[879, 622, 291]"
1,StoreType,"[d, a]"
2,Assortment,"[a, c]"
3,CompetitionDistance,[]
4,CompetitionOpenSinceMonth,[1.0]
5,CompetitionOpenSinceYear,[1900.0]
6,Promo2,"[0, 1]"
7,Promo2SinceWeek,"[0.0, 5.0]"
8,Promo2SinceYear,"[0.0, 2013.0]"
9,PromoInterval,"[0,0,0,0, Feb,May,Aug,Nov]"


All rows with missing values in the column CompetitionDistance has a value of 1 in the column CompetitionBeforeStoreOpened. But this doesn't give us much information why the CompetitionDistance is null. But from common sense I guess this is because the competitions for these stores are very far away and have almost no impact on these stores that they weren't measured when collecting the data. For this reason, I will impute this values with the maximum competition distance.

In [171]:
max_dist = store_df['CompetitionDistance'].max()
max_dist

75860.0

In [172]:
fix_missing_value(store_df, ['CompetitionDistance'], max_dist)

3 missing values in the column CompetitionDistance have been replaced by 75860.0.


In [173]:
# final check for missing values
percent_missing_values(store_df)

The dataset contains 0.0 % missing values.


### train_df

In [174]:
percent_missing_values(train_df)

The dataset contains 0.0 % missing values.


We are lucky to have no missing values in this dataset as it contains more than a million rows. This is a rare thing to happen in the real world.

## Data Types

### store_df

Let's check if there are columns with mixed data types.

In [175]:
show_cols_mixed_dtypes(store_df)

None of the columns contain mixed types.


In [176]:
store_df.dtypes

Store                             int64
StoreType                        object
Assortment                       object
CompetitionDistance             float64
CompetitionOpenSinceMonth       float64
CompetitionOpenSinceYear        float64
Promo2                            int64
Promo2SinceWeek                 float64
Promo2SinceYear                 float64
PromoInterval                    object
CompetitionBeforeStoreOpened      int64
dtype: object

In [177]:
store_df.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,CompetitionBeforeStoreOpened
0,1,c,a,1270.0,9.0,2008.0,0,0.0,0.0,"0,0,0,0",0
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",0
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",0
3,4,c,c,620.0,9.0,2009.0,0,0.0,0.0,"0,0,0,0",0
4,5,a,a,29910.0,4.0,2015.0,0,0.0,0.0,"0,0,0,0",0


As we can see from the dataframe above StoreType, Assortment and PromoInterval hold string values. Note that even though PromoInterval holds list values, the are enclosed in string literals. We will convert the data types of these three columns to string. Converting PromoInterval to string is convenient for further processing elements in each value.

In [178]:
# get the columns with object data type
string_columns = store_df.select_dtypes(include='object').columns.tolist()
string_columns

['StoreType', 'Assortment', 'PromoInterval']

In [179]:
convert_to_string(store_df, string_columns)

We also know that the columns CompetitionOpenSinceMonth, CompetitionOpenSinceYear, Promo2SinceWeek, and Promo2SinceYear contain week, month and year information which can't be float. So we will change the data type of these columns to int64.

In [180]:
convert_to_int(store_df, ['CompetitionOpenSinceMonth',  'CompetitionOpenSinceYear',
        'Promo2SinceWeek', 'Promo2SinceYear'])

In [181]:
store_df.dtypes

Store                             int64
StoreType                        string
Assortment                       string
CompetitionDistance             float64
CompetitionOpenSinceMonth         int64
CompetitionOpenSinceYear          int64
Promo2                            int64
Promo2SinceWeek                   int64
Promo2SinceYear                   int64
PromoInterval                    string
CompetitionBeforeStoreOpened      int64
dtype: object

In [182]:
store_df.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,CompetitionBeforeStoreOpened
0,1,c,a,1270.0,9,2008,0,0,0,"0,0,0,0",0
1,2,a,a,570.0,11,2007,1,13,2010,"Jan,Apr,Jul,Oct",0
2,3,a,a,14130.0,12,2006,1,14,2011,"Jan,Apr,Jul,Oct",0
3,4,c,c,620.0,9,2009,0,0,0,"0,0,0,0",0
4,5,a,a,29910.0,4,2015,0,0,0,"0,0,0,0",0


### train_df

Let's check if there are columns with mixed data types.

In [183]:
show_cols_mixed_dtypes(train_df)

         Column      Data type
0  StateHoliday  mixed-integer


So StateHoliday needs more investigation of the datatypes it contain. Let's look at the unique values.

In [184]:
train_df['StateHoliday'].value_counts()

0    855087
0    131072
a     20260
b      6690
c      4100
Name: StateHoliday, dtype: int64

In [185]:
train_df['StateHoliday'].value_counts().index

Index(['0', 0, 'a', 'b', 'c'], dtype='object')

Here 0 is represented as an integer and a string. Since there are other string values we will convert the  StateHoliday column into string.

In [186]:
convert_to_string(train_df, ['StateHoliday'])

In [187]:
train_df.dtypes

Store             int64
DayOfWeek         int64
Date             object
Sales             int64
Customers         int64
Open              int64
Promo             int64
StateHoliday     string
SchoolHoliday     int64
dtype: object

In [188]:
train_df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


By observing the columns we can realize that the Date column actually contain datetime values even though its data type is object. So we will convert it to datetime data type.

In [189]:
convert_to_datetime(train_df, ['Date'])

In [190]:
train_df.dtypes

Store                     int64
DayOfWeek                 int64
Date             datetime64[ns]
Sales                     int64
Customers                 int64
Open                      int64
Promo                     int64
StateHoliday             string
SchoolHoliday             int64
dtype: object

## Duplicates

### store_df

In [191]:
# search for duplicate rows and drop them
drop_duplicates(store_df)

No duplicate rows were found.


Since store is the id that uniquely identifies each record, there should be no duplicates. Let's check that.

In [192]:
store_df.duplicated(subset=['Store']).all()

False

We are safe. No two rows in our dataset contain the same Store value.

### train_df

In [193]:
# search for duplicate rows and drop them
drop_duplicates(train_df)

No duplicate rows were found.


Since Store and Date are the columns that uniquely identify each record, there should be no duplicates. We can't have duplicate data for a store with the same date. Let's check whether such cases exist.

In [194]:
train_df.duplicated(subset=['Store', 'Date']).all()

False

We are safe. No two rows in our dataset contain the same combination of Store and Date values.

## Feature Engineering

### train_df

Even though we have a date column it will be complex to query our data based on week, month, year, etc. So inorder tp gain complex insights easily, I have derived the following features.

In [195]:
train_df['Year'] = train_df['Date'].apply(lambda x: x.year)
train_df['Month'] = train_df['Date'].apply(lambda x: x.month)
train_df['DayOfMonth'] = train_df['Date'].apply(lambda x: x.day)
train_df['WeekOfYear'] = train_df['Date'].apply(lambda x: x.weekofyear)
train_df['weekday'] = train_df['DayOfWeek'].apply(lambda x: 1 if (x in [5, 6]) else 0)

In [196]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1017209 entries, 0 to 1017208
Data columns (total 14 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   Store          1017209 non-null  int64         
 1   DayOfWeek      1017209 non-null  int64         
 2   Date           1017209 non-null  datetime64[ns]
 3   Sales          1017209 non-null  int64         
 4   Customers      1017209 non-null  int64         
 5   Open           1017209 non-null  int64         
 6   Promo          1017209 non-null  int64         
 7   StateHoliday   1017209 non-null  string        
 8   SchoolHoliday  1017209 non-null  int64         
 9   Year           1017209 non-null  int64         
 10  Month          1017209 non-null  int64         
 11  DayOfMonth     1017209 non-null  int64         
 12  WeekOfYear     1017209 non-null  int64         
 13  weekday        1017209 non-null  int64         
dtypes: datetime64[ns](1), int64(12), s

In [197]:
train_df.sample(10)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,DayOfMonth,WeekOfYear,weekday
699512,78,6,2013-10-12,2753,448,1,0,0,0,2013,10,12,41,1
437216,922,4,2014-06-05,7597,690,1,1,0,0,2014,6,5,23,0
542018,914,1,2014-03-03,14334,1515,1,1,0,0,2014,3,3,10,0
539556,682,3,2014-03-05,14197,2003,1,1,0,0,2014,3,5,10,0
866306,737,4,2013-05-16,5800,917,1,1,0,0,2013,5,16,20,0
181270,641,4,2015-02-19,6788,564,1,1,0,0,2015,2,19,8,0
558795,966,7,2014-02-16,0,0,0,0,0,0,2014,2,16,7,0
222485,601,2,2015-01-13,5052,383,1,1,0,0,2015,1,13,3,0
334495,1045,4,2014-09-18,7335,1003,1,1,0,0,2014,9,18,38,0
933530,1061,7,2013-03-17,0,0,0,0,0,0,2013,3,17,11,0


## Saving Data

In [198]:
# save the clean dataframe to a csv file
train_df.to_csv('../data/clean_train.csv')
store_df.to_csv('../data/clean_store.csv')